import numpy as np
import pandas as pd
import tqdm
import argparse
import os
import shutil
from ai_clinician.preprocessing.utils import load_csv
from ai_clinician.preprocessing.columns import *
from ai_clinician.modeling.models.komorowski_model import AIClinicianModel
from ai_clinician.modeling.models.common import *
from ai_clinician.modeling.models.dqn import DuelingDQNModel
from ai_clinician.modeling.columns import C_OUTCOME
import pickle
from sklearn.model_selection import train_test_split

tqdm.tqdm.pandas()

In [1]:
import numpy as np 
import pandas as pd 
import tqdm 
import argparse 
import os 
import shutil 
from ai_clinician.preprocessing.utils import load_csv 
from ai_clinician.preprocessing.columns import * 
from ai_clinician.modeling.models.komorowski_model import AIClinicianModel 
from ai_clinician.modeling.models.common import * 
from ai_clinician.modeling.models.dqn import DuelingDQNModel 
from ai_clinician.modeling.columns import C_OUTCOME 
import pickle 
from sklearn.model_selection import train_test_split

tqdm.tqdm.pandas()

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('/home/lkapral/RRT_mimic_iv/data/mimic/mimic_dataset.csv')

In [4]:
import argparse
import os
import shutil
import pandas as pd

# Function to create and parse arguments
def create_args():
    parser = argparse.ArgumentParser(description='Simulate command-line argument parsing in Jupyter notebook.')
    parser.add_argument('data', type=str,
                        help='Model data directory (should contain train and test directories)')
    parser.add_argument('--worker-label', dest='worker_label', type=str, default='',
                        help='Label to suffix output files')
    parser.add_argument('--save', dest='save_behavior', type=str, default='best',
                        help='Models to save (best [default], all, none)')
    parser.add_argument('--val-size', dest='val_size', type=float, default=0.2,
                        help='Proportion of data to use for validation')
    parser.add_argument('--n-models', dest='n_models', type=int, default=500,
                        help='Number of models to build')
    parser.add_argument('--model-type', dest='model_type', type=str, default='AIClinician',
                        help='Model type to train (AIClinician or DuelingDQN)')
    parser.add_argument('--cluster-fraction', dest='cluster_fraction', type=float, default=0.25,
                        help='Fraction of patient states to sample for state clustering')
    parser.add_argument('--n-cluster-init', dest='n_cluster_init', type=int, default=32,
                        help='Number of cluster initializations to try in each replicate')
    parser.add_argument('--n-cluster-states', dest='n_cluster_states', type=int, default=500,
                        help='Number of states to define through clustering')
    parser.add_argument('--n-action-bins', dest='n_action_bins', type=int, default=5,
                        help='Number of action bins for fluids and vasopressors')
    parser.add_argument('--reward', dest='reward', type=int, default=100,
                        help='Value to assign as positive reward if discharged from hospital, or negative reward if died')
    parser.add_argument('--transition-threshold', dest='transition_threshold', type=int, default=5,
                        help='Prune state-action pairs with less than this number of occurrences in training data')
    parser.add_argument('--gamma', dest='gamma', type=float, default=0.99,
                        help='Decay for reward values (default 0.99)')
    parser.add_argument('--soften-factor', dest='soften_factor', type=float, default=0.01,
                        help='Amount by which to soften factors (random actions will be chosen this proportion of the time)')
    parser.add_argument('--num-iter-ql', dest='num_iter_ql', type=int, default=6,
                        help='Number of bootstrappings to use for TD learning (physician policy)')
    parser.add_argument('--num-iter-wis', dest='num_iter_wis', type=int, default=700,
                        help='Number of bootstrappings to use for WIS estimation (AI policy)')
    parser.add_argument('--state-dim', dest='state_dim', type=int, default=256,
                        help='Dimension for learned state representation in DQN')
    parser.add_argument('--hidden-dim', dest='hidden_dim', type=int, default=128,
                        help='Number of units in hidden layer for DQN')

    # Simulate input arguments as if they were passed from the command line
    simulated_input = '--n-models 500 --model-type AIClinician --n-action-bins 2 --val-size 0.2'.split()
    simulated_input.insert(0, '/home/lkapral/RRT_mimic_iv/data/model')
    return parser.parse_args(simulated_input)

# Create args object
args = create_args()

# Now, the rest of your script can use these args as if they were passed from the command line
data_dir = args.data
worker_label = args.worker_label
n_models = args.n_models
model_type = args.model_type
n_action_bins = args.n_action_bins
fixed_num_features = 40
# and so on for other arguments

# You can now use these variables in your code to set up directories, load data, etc.


# Loading CSV files
MIMICraw = pd.read_csv(os.path.join(data_dir, "train", "MIMICraw.csv"))
MIMICzs = pd.read_csv(os.path.join(data_dir, "train", "MIMICzs.csv"))
metadata = pd.read_csv(os.path.join(data_dir, "train", "metadata.csv"))
unique_icu_stays = metadata['icustayid'].unique()




In [5]:
feature_importance = pd.read_csv('/home/lkapral/RRT_mimic_iv/data/model/combined_feature_importances.csv')

weights = feature_importance.head(fixed_num_features)['Combined_Average'].values
feature_weights = weights / np.linalg.norm(weights)

In [6]:
reduced_features = feature_importance.head(fixed_num_features)['Feature'].tolist()

In [7]:
len(unique_icu_stays)

37992

In [8]:
metadata.columns

Index(['bloc', 'icustayid', 'outcome'], dtype='object')

In [9]:
MIMICraw

,gender,mechvent,extubated,max_dose_vaso,re_admission,APACHEII_Renal_Failure,age,Height_cm,Weight_kg,GCS,RASS,HR,SysBP,MeanBP,DiaBP,RR,Temp_C,CVP,PAPsys,PAPmean,PAPdia,CI,FiO2_1,O2flow,PEEP,TidalVolume,MinuteVentil,PAWmean,PAWpeak,PAWplateau,Potassium,Sodium,Chloride,Glucose,Magnesium,Calcium,Ionised_Ca,Total_protein,Albumin,Troponin,Hb,Ht,RBC_count,WBC_count,Platelets_count,PTT,PT,ACT,Arterial_pH,paO2,paCO2,Arterial_BE,HCO3,ETCO2,Arterial_lactate,SvO2,Anion_Gap,Fibrinogen,Absolute_Neutrophil_Count,Phosphorous,SaO2,LDH,T3,Osmolality_Urine,SOFA,SIRS,Shock_Index,PaO2_FiO2,cumulated_balance,Blood_Flow,Hourly_Patient_Fluid_Removal,Dialysate_Rate,Ultrafiltrate_Output,Potassium_Urine,Sodium_Urine,Urea_Nitrogen_Urine,Heparin_LMW,Creatinine_Urine,SpO2,BUN,Creatinine,SGOT,SGPT,Total_bili,INR,CRP,Ammonia,Triglyceride,CK_MB,BNP,Thyroid_Stimulating_Hormone,Hemodialysis_Output,Citrate,Prefilter_Replacement_Rate,Postfilter_Replacement_Rate,Iron,Gamma_Glutamyltransferase,input_total,input_step,output_total,output_step
0,0,0,NaN,0.000,True,NaN,92,163.0,68.5,14.500000,-0.500000,88.651550,111.950000,75.700000,57.575000,15.522750,36.277800,12.214300,NaN,NaN,NaN,NaN,0.340000,3.500000,7.500000,732.625000,9.900000,9.500000,13.000000,19.333350,4.000000,142.500000,102.500000,93.000000,2.05000,8.550000,1.100000,NaN,2.75,0.135000,10.750000,33.350000,3.135000,12.300000,63.000000,82.525650,14.800000,NaN,7.448000,77.900000,46.900000,-3.000000,23.500000,NaN,1.110000,NaN,11.000000,349.000000,NaN,2.750000,NaN,522.500000,NaN,NaN,6,2,0.813359,240.138900,-110.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.000000,15.000000,0.500000,30.500000,86.500000,1.000000,1.30000,56.5,NaN,NaN,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.00,250.00,360.0,360.0
1,0,0,NaN,0.000,True,NaN,92,163.0,68.5,15.000000,0.000000,86.027767,99.210333,60.130967,40.591267,14.277767,35.688300,7.916667,NaN,NaN,NaN,NaN,0.360000,4.000000,5.000000,495.000000,7.275000,8.666667,24.333333,25.041667,5.033333,137.333333,104.666667,125.333333,2.20000,7.633333,1.133333,NaN,2.50,0.205000,8.333333,30.566667,2.748890,25.100000,280.666667,35.866667,15.500000,NaN,7.480000,21.000000,59.000000,10.000000,25.000000,NaN,1.844443,NaN,12.333333,438.500000,NaN,2.366667,NaN,411.333333,NaN,NaN,7,2,0.871841,58.333300,2044.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.793667,46.000000,1.400000,42.000000,147.888867,0.566667,1.35000,56.5,NaN,NaN,26.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3754.85,1504.85,1710.0,320.0
2,0,0,NaN,0.150,True,NaN,92,163.0,68.5,15.000000,0.000000,92.033333,116.289000,67.250000,46.063900,15.327767,36.390700,6.666667,NaN,NaN,NaN,NaN,0.360000,4.000000,5.000000,412.166667,7.337500,8.000000,20.916667,20.416667,5.233333,136.000000,104.000000,94.000000,2.30000,7.800000,1.216667,NaN,2.50,0.230000,8.100000,24.600000,3.082223,24.200000,357.000000,36.100000,16.200000,NaN,7.456667,21.000000,59.000000,2.333333,27.000000,NaN,1.533333,NaN,10.000000,676.666667,NaN,1.900000,NaN,419.000000,NaN,NaN,6,2,0.794110,58.333300,3048.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.800000,47.000000,1.200000,50.000000,32.333300,0.300000,1.40000,56.5,NaN,NaN,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6488.55,678.00,3440.0,620.0
3,0,0,NaN,0.048,True,NaN,92,163.0,68.5,14.714300,-0.285714,96.530567,126.625000,82.905567,61.045833,20.269433,36.608300,7.533333,NaN,NaN,NaN,NaN,0.360000,4.000000,6.000000,437.733333,8.647777,7.666667,12.622233,16.000000,4.811110,137.333333,104.666667,108.074000,2.30000,8.077777,1.113333,5.2,2.70,0.213333,8.700000,25.822233,3.313333,21.533333,343.666667,39.155567,16.088900,NaN,7.463333,136.000000,37.000000,4.333333,25.666667,NaN,1.766667,NaN,10.000000,842.000000,NaN,2.011110,NaN,308.444333,NaN,NaN,2,3,0.762922,377.778000,2353.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,43.666667,1.255557,38.333333,20.666667,0.355556,1.40000,56.5,NaN,NaN,7.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7583.55,95.00,5230.0,610.0
4,0,0,NaN,0.000,True,NaN,92,163.0,68.5,14.714300,-0.285714,90.483333,117.816667,77.816667,54.416667,19.733333,

In [10]:
print("Create actions")

# List of RRT-related columns
rrt_cols = [
    'Ultrafiltrate_Output',
    'Blood_Flow',
    'Hourly_Patient_Fluid_Removal',
    'Dialysate_Rate',
    'Hemodialysis_Output',
    'Citrate',
    'Prefilter_Replacement_Rate',
    'Postfilter_Replacement_Rate'
]

# Create a boolean mask where any RRT column is not NaN and not zero
rrt_actions = (~MIMICraw[rrt_cols].isna() & (MIMICraw[rrt_cols] != 0)).any(axis=1)
MIMICraw['action'] = rrt_actions.astype(int)

# Update the number of actions
n_actions = 2  # Actions are now binary: 0 or 1

# Simplified fit_action_bins function for binary actions
def fit_action_bins_binary(actions):
    action_medians = np.array([0, 1])
    action_bins = np.array([0, 0.5, 1])
    all_actions = actions.values
    return all_actions, action_medians, action_bins

# Create all_actions, action_medians, action_bins
all_actions, action_medians, action_bins = fit_action_bins_binary(MIMICraw['action'])



model_type = args.model_type

np.seterr(divide='ignore', invalid='ignore')


Create actions


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [11]:
MIMICraw['RRT'] = MIMICraw['action']
MIMICzs['RRT'] = MIMICraw['action']

In [12]:
MIMICzs = MIMICzs[reduced_features+ ['RRT']]
MIMICraw = MIMICraw[reduced_features + ['action']]
feature_weights = np.append(feature_weights,1)

In [13]:
MIMICraw['icustayid'] = metadata['icustayid']
len(MIMICraw[MIMICraw['action']>0]['icustayid'].unique())

/tmp/ipykernel_2355364/2236096699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MIMICraw['icustayid'] = metadata['icustayid']


1457

In [14]:
MIMICraw['RRT'] = MIMICraw['action']
MIMICzs['RRT'] = MIMICraw['action']

# MIMICraw['icustayid'] = metadata['icustayid']

# MIMICraw['action'] = MIMICraw.groupby('icustayid')['action'].transform(
#     lambda x: x.ne(x.shift().fillna(x)).astype(int)
# )
# MIMICraw.drop(columns=['icustayid'], inplace=True)



/tmp/ipykernel_2355364/2285299192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MIMICraw['RRT'] = MIMICraw['action']


In [15]:
print("Action distribution:")
print(MIMICraw['action'].value_counts())

print("Action medians:")
print(action_medians)

print("Action bins:")
print(action_bins)


Action distribution:
action
0    281671
1     11423
Name: count, dtype: int64
Action medians:
[0 1]
Action bins:
[0.  0.5 1. ]


In [16]:
args.n_models

500

In [17]:
def run_penalty_experiment(
    number_of_models,
    penal_amount, fixed_num_features,
    X_train, X_val, metadata_train, metadata_val, actions_train, actions_val,
    data_dir, args, model_type, AIClinicianModel, DuelingDQNModel,
    n_action_bins, action_bins, action_medians, 
    feature_weights, build_complete_record_sequences,
    evaluate_physician_policy_td, evaluate_policy_wis,
    train_ids, val_ids
):

    out_dir = os.path.join(data_dir, "models_penal", str(penal_amount))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    model_specs_dir = os.path.join(out_dir, f"model_params_{fixed_num_features}")
    if not os.path.exists(model_specs_dir):
        os.makedirs(model_specs_dir)

    all_model_stats = []
    max_wis_lb = 0   # Save the best model with value > 0
    min_wis_lb = 1e9 # Also save the worst model
    
    # Loop over the number of models to train per penalty value
    for modl in range(args.n_models):
        print(f"Penalty: {penal_amount}, Model {modl} of {number_of_models}")

        base_model = AIClinicianModel(
            n_cluster_states=args.n_cluster_states,
            n_actions=n_actions,
            cluster_fit_fraction=args.cluster_fraction,
            n_cluster_init=args.n_cluster_init,
            gamma=args.gamma,
            reward_val=args.reward,
            transition_threshold=args.transition_threshold,
            penalise_action_amount=penal_amount
        )

        if model_type == 'DuelingDQN':
            model = DuelingDQNModel(
                state_dim=args.state_dim,
                n_actions=n_actions,
                hidden_dim=args.hidden_dim,
                gamma=args.gamma,
                reward_val=args.reward
            )
        else:
            model = base_model

        # Train the base model
        base_model.train(
            X_train.values,
            actions_train,
            metadata_train,
            X_val=X_val.values,
            actions_val=actions_val,
            metadata_val=metadata_val,
            feature_weights=feature_weights,
        )

        # If a specialized model is used (e.g. DuelingDQN), train it too
        if model != base_model:
            model.train(
                X_train.values,
                actions_train,
                metadata_train,
                X_val=X_val.values,
                actions_val=actions_val,
                metadata_val=metadata_val
            )

        ####### EVALUATE ON TRAIN SET #######
        states_train = base_model.compute_states(X_train.values)
        print("Evaluate on MIMIC training set")
        records_train = build_complete_record_sequences(
            metadata_train,
            states_train,
            actions_train,
            base_model.absorbing_states,
            base_model.rewards
        )

        train_bootql = evaluate_physician_policy_td(
            records_train,
            base_model.physician_policy,
            args.gamma,
            args.num_iter_ql,
            args.n_cluster_states
        )

        phys_probs_train = base_model.compute_physician_probabilities(states=states_train, actions=actions_train)
        model_probs_train = model.compute_probabilities(X=X_train.values, actions=actions_train)
        train_bootwis, _, _ = evaluate_policy_wis(
            metadata_train,
            phys_probs_train,
            model_probs_train,
            base_model.rewards,
            args.gamma,
            args.num_iter_wis
        )

        model_stats = {}
        model_stats['train_bootql_mean'] = np.nanmean(train_bootql)
        model_stats['train_bootql_0.99'] = np.quantile(train_bootql, 0.99)
        model_stats['train_bootql_0.95'] = np.quantile(train_bootql, 0.95)
        model_stats['train_bootwis_mean'] = np.nanmean(train_bootwis)
        model_stats['train_bootwis_0.05'] = np.quantile(train_bootwis, 0.05)
        model_stats['train_bootwis_0.95'] = np.quantile(train_bootwis, 0.95)

        ####### EVALUATE ON VALIDATION SET #######
        print("Evaluate on MIMIC validation set")
        states_val = base_model.compute_states(X_val.values)

        records_val = build_complete_record_sequences(
            metadata_val,
            states_val,
            actions_val,
            base_model.absorbing_states,
            base_model.rewards
        )

        val_bootql = evaluate_physician_policy_td(
            records_val,
            base_model.physician_policy,
            args.gamma,
            args.num_iter_ql,
            args.n_cluster_states
        )

        phys_probs_val = base_model.compute_physician_probabilities(states=states_val, actions=actions_val)
        phys_probs_all_val = base_model.compute_physician_probabilities(states=states_val)
        model_probs_val = model.compute_probabilities(X=X_val.values, actions=actions_val)
        model_probs_all_val = model.compute_probabilities(X=X_val.values)

        val_bootwis, _, _ = evaluate_policy_wis(
            metadata_val,
            phys_probs_val,
            model_probs_val,
            base_model.rewards,
            args.gamma,
            args.num_iter_wis
        )

        model_stats['val_bootql_0.95'] = np.quantile(val_bootql, 0.95)
        model_stats['val_bootql_mean'] = np.nanmean(val_bootql)
        model_stats['val_bootql_0.99'] = np.quantile(val_bootql, 0.99)
        model_stats['val_bootwis_mean'] = np.nanmean(val_bootwis)
        model_stats['val_bootwis_0.01'] = np.quantile(val_bootwis, 0.01)
        wis_95lb = np.quantile(val_bootwis, 0.05)
        model_stats['val_bootwis_0.05'] = wis_95lb
        model_stats['val_bootwis_0.95'] = np.quantile(val_bootwis, 0.95)
        print("95% LB: {:.2f}".format(wis_95lb))

        all_model_stats.append(model_stats)

        best = False
        worst = False
        if wis_95lb > max_wis_lb:
            best = True
            max_wis_lb = wis_95lb
        if wis_95lb < min_wis_lb:
            worst = True
            min_wis_lb = wis_95lb

        save_name = None
        if args.save_behavior == 'best' and best:
            save_name = 'best_model'
        elif args.save_behavior == 'best' and worst:
            save_name = 'worst_model'
        elif args.save_behavior == 'all':
            save_name = 'model_' + str(modl)
        if save_name:
            print("Saving model:", save_name)
            save_path = os.path.join(
                model_specs_dir,
                '{}.pkl'.format(save_name))
            model.save(save_path, metadata={
                'actions': {
                    'n_action_bins': n_action_bins,
                    'action_bins': action_bins,
                    'action_medians': action_medians
                },
                'split': {
                    'train_ids': train_ids,
                    'val_ids': val_ids,
                },
                'eval': {
                    'num_iter_ql': args.num_iter_ql,
                    'num_iter_wis': args.num_iter_wis
                },
                'model_num': modl
            })

            # Save additional variables
            additional_vars = {
                'states_val': states_val,
                'X_val': X_val.values,
                'actions_val': actions_val,
                'metadata_val': metadata_val,
                'phys_probs': phys_probs_all_val,
                'model_probs': model_probs_all_val,
                'rewards': base_model.rewards,
                'gamma': args.gamma
            }

            with open(os.path.join(out_dir, 'additional_vars.pkl'), 'wb') as f:
                pickle.dump(additional_vars, f)

        # Save stats
        all_model_stats_df = pd.DataFrame(all_model_stats)
        stats_file = os.path.join(out_dir, "model_stats{}.csv".format('_' + args.worker_label if args.worker_label else ''))
        all_model_stats_df.to_csv(stats_file, float_format='%.6f')

    print(f'Penalty {penal_amount}: Done.')
    return penal_amount, all_model_stats


In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

train_ids, val_ids = train_test_split(unique_icu_stays, test_size=args.val_size)
train_indexes = metadata[metadata[C_ICUSTAYID].isin(train_ids)].index
val_indexes = metadata[metadata[C_ICUSTAYID].isin(val_ids)].index

X_train = MIMICzs.iloc[train_indexes]
X_val = MIMICzs.iloc[val_indexes]
metadata_train = metadata.iloc[train_indexes]
metadata_val = metadata.iloc[val_indexes]
actions_train = all_actions[train_indexes]
actions_val = all_actions[val_indexes]


penal_amounts = [i/100. for i in range(-1, 15, 1)]
results = Parallel(n_jobs=len(penal_amounts)+1)(
    delayed(run_penalty_experiment)(
        args.n_models,
        penal_amount, 
        fixed_num_features,
        X_train, X_val, metadata_train, metadata_val, actions_train, actions_val,
        data_dir, args, model_type, AIClinicianModel, DuelingDQNModel,
        n_action_bins, action_bins, action_medians, 
        feature_weights, build_complete_record_sequences,
        evaluate_physician_policy_td, evaluate_policy_wis,
        train_ids, val_ids
    )
    for penal_amount in penal_amounts)


/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/common.py:142: RuntimeWarning: invalid value encountered in divide
  transitionr = np.nan_to_num(np.where(transitionr > 0, transitionr / action_counts, 0))
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/common.py:143: RuntimeWarning: invalid value encountered in divide
  physpol = np.nan_to_num(action_counts / action_counts.sum(axis=1, keepdims=True))
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/common.py:142: RuntimeWarning: invalid value encountered in divide
  transitionr = np.nan_to_num(np.where(transitionr > 0, transitionr / action_counts, 0))
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/common.py:143: RuntimeWarning: invalid value encountered in divide
  physpol = np.nan_to_num(action_counts / action_counts.sum(axis=1, keepdims=True))
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/common.py:142: RuntimeWarning: invalid value encountered in divide
  transitionr = np.nan_to_num(np.where(

Penalty: -0.01, Model 0 of 500
Clustering
Zeroing out 433/1004 transitions
Create reward matrix R(S, A)
Policy iteration
Evaluate on MIMIC training set


TD evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

Penalty: 0.11, Model 0 of 500
Clustering
Zeroing out 418/1004 transitions
Create reward matrix R(S, A)
Policy iteration
Evaluate on MIMIC training set


TD evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

...

TD evaluation:  17%|█▋        | 1/6 [00:02<00:11,  2.21s/it]

95% LB: 76.69
Saving model: best_model
Penalty: 0.14, Model 1 of 500
Clustering
Zeroing out 414/1004 transitions
Create reward matrix R(S, A)
Policy iteration
Evaluate on MIMIC training set


TD evaluation:   0%|          | 0/6 [00:00<?, ?it/s].23s/it]

95% LB: 77.66
Saving model: best_model
Penalty: 0.09, Model 1 of 500
Clustering
Zeroing out 415/1004 transitions
Create reward matrix R(S, A)
Policy iteration
Evaluate on MIMIC training set


TD evaluation: 100%|██████████| 6/6 [00:13<00:00,  2.21s/it]
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/komorowski_model.py:339: RuntimeWarning: divide by zero encountered in divide
  z = self.soften_factor / ii.sum()
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/komorowski_model.py:342: RuntimeWarning: divide by zero encountered in scalar divide
  soft_physpol[i, ~ii] = soft_physpol[i,~ii] * (1 - self.soften_factor / coef)
TD evaluation: 100%|██████████| 6/6 [00:13<00:00,  2.20s/it]
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/komorowski_model.py:339: RuntimeWarning: divide by zero encountered in divide
  z = self.soften_factor / ii.sum()
TD evaluation: 100%|██████████| 6/6 [00:12<00:00,  2.17s/it]
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/komorowski_model.py:339: RuntimeWarning: divide by zero encountered in divide
  z = self.soften_factor / ii.sum()
/home/lkapral/RRT_mimic_iv/ai_clinician/modeling/models/komorowski_model.py:342: Runtim

In [ ]:
len(penal_amounts)